In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%aimport deepsvr

Populating the interactive namespace from numpy and matplotlib


In [2]:
%matplotlib inline

In [6]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle

from analysis_utils.ClassifierPlots import create_reliability_diagram, create_roc_curve, create_feature_importance_plot
from analysis_utils.Analysis import print_accuracy_and_classification_report, predict_classes, get_somatic_error_type

sns.set_style("white")
sns.set_context('poster')

In [4]:
# Pull in training data
training_data = pd.read_pickle('train_val_test_sets/train_methyl_array.pkl')
validation_data = pd.read_pickle('train_val_test_sets/val_methyl_array.pkl')
testing_data = pd.read_pickle('train_val_test_sets/test_methyl_array.pkl')

print("\n#### Training data:")
print(type(training_data))
print(type(training_data['pheno']), type(training_data['beta']))
print(training_data['beta'].columns)
print(training_data['beta'].shape)
print(training_data['pheno'].columns)
print(training_data['pheno'].shape)


print("\n#### Validation data:")
print(type(validation_data))
print(type(validation_data['pheno']), type(validation_data['beta']))
print(validation_data['beta'].columns)
print(validation_data['beta'].shape)
print(validation_data['pheno'].columns)
print(validation_data['pheno'].shape)

print("\n#### Testing data:")
print(type(testing_data))
print(type(testing_data['pheno']), type(testing_data['beta']))
print(testing_data['beta'].columns)
print(testing_data['beta'].shape)
print(testing_data['pheno'].columns)
print(testing_data['pheno'].shape)

# print(training_data)
# training_data.sort_index(axis=1, inplace=True)


#### Training data:
<class 'dict'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
Index(['cg14112997', 'cg02368820', 'cg16464924', 'cg11738485', 'cg19697575',
       'cg24007926', 'cg11956442', 'cg10890644', 'cg00540295', 'cg14061270',
       ...
       'cg24575128', 'cg02575448', 'cg14315334', 'cg10715905', 'cg12298697',
       'cg06890747', 'cg11282353', 'cg25381017', 'cg07224147', 'cg01498829'],
      dtype='object', length=25000)
(176, 25000)
Index(['X', 'Basename', 'AccNum', 'disease', 'Age', 'Sex', 'Tissue', 'Gran',
       'CD4T', 'CD8T', 'Bcell', 'Mono', 'NK', 'gMDSC', 'filenames'],
      dtype='object')
(176, 15)

#### Validation data:
<class 'dict'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
Index(['cg14112997', 'cg02368820', 'cg16464924', 'cg11738485', 'cg19697575',
       'cg24007926', 'cg11956442', 'cg10890644', 'cg00540295', 'cg14061270',
       ...
       'cg24575128', 'cg02575448', 'cg14315334', 'cg10715905',

In [5]:
# Show the calls associated with training data
# print(training_data['pheno'].groupby('Age').size())
# print(training_data['pheno'].groupby('NK').size())
# print(training_data['pheno'].groupby('CD8T').size())
# print(training_data['pheno'].groupby('gMDSC').size())
training_data['beta'].columns.values

array(['cg14112997', 'cg02368820', 'cg16464924', ..., 'cg25381017',
       'cg07224147', 'cg01498829'], dtype=object)

In [6]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Age Regressor

In [7]:
# Get training data as numpy array
X_train = training_data['beta']
X_train = X_train.reset_index().drop(columns=['index']) # remove the sample-name indices
X_train = X_train.values
print(X_train)

# Get labels for training data
Y_train = training_data['pheno'].Age
Y_train = Y_train.reset_index().drop(columns=['index']) # remove the sample-name indices
Y_train = Y_train.values.reshape(1, -1)[0]
print(Y_train)

# Get test data as numpy array
X_test = validation_data['beta']
X_test = X_test.reset_index().drop(columns=['index']) # remove the sample-name indices
X_test = X_test.values
print(X_test)

# Get labels for test data
Y_test = validation_data['pheno'].Age
Y_test = Y_test.reset_index().drop(columns=['index']) # remove the sample-name indices
Y_test = Y_test.values.reshape(1, -1)[0]
print(Y_test)

# Get validation data as numpy array
X_val = testing_data['beta']
X_val = X_val.reset_index().drop(columns=['index']) # remove the sample-name indices
X_val = X_val.values
print(X_val)

# Get labels for validation data
Y_val = testing_data['pheno'].Age
Y_val = Y_val.reset_index().drop(columns=['index']) # remove the sample-name indices
Y_val = Y_val.values.reshape(1, -1)[0]
print(Y_val)

[[0.74677163 0.13775374 0.88551003 ... 0.48341863 0.66089444 0.51689468]
 [0.04887406 0.90537354 0.97906541 ... 0.49703332 0.66324587 0.58863093]
 [0.72727367 0.14917133 0.89311984 ... 0.50181661 0.66569283 0.56854207]
 ...
 [0.04744683 0.956407   0.92370621 ... 0.53528889 0.62721045 0.59115508]
 [0.21039367 0.90989842 0.36555745 ... 0.54765308 0.84242357 0.50499059]
 [0.81269162 0.14452031 0.20225115 ... 0.45019963 0.74177941 0.5883739 ]]
[26. 16. 47. 33. 91. 15. 16. 17. 15. 16. 59. 49. 40. 54. 31. 41. 39. 25.
 62. 69. 46. 31. 79. 49. 55. 82. 20. 39. 51. 61. 60. 58. 70. 16. 59. 30.
 62. 50. 56. 79. 21. 40. 70. 72. 69. 76. 19. 53. 82. 53. 32. 59. 55. 16.
 74. 52. 55. 72. 71. 44. 40. 65. 36. 43. 67. 52. 53. 45. 47. 16. 53. 44.
 58. 44. 19. 64. 37. 53. 34. 77. 59. 49. 75. 35. 44. 16. 44. 51. 69. 19.
 68. 45. 19. 83. 45. 24. 71. 16. 82. 80. 42. 69. 45. 35. 62. 78. 83. 19.
 19. 15. 31. 76. 27. 22. 70. 15. 36. 69. 24. 15. 77. 84. 44. 17. 44. 86.
 76. 35. 40. 72. 45. 49. 47. 41. 23. 80. 15. 

In [11]:
# Set parameters kfold
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)

In [12]:
# Initialize Estimator
estimator = LinearRegression(n_jobs=-1)

In [13]:
# Perform cross validation
probabilities_train = cross_val_predict(estimator, X_train, Y_train, cv=kfold, method='predict')

probabilities_val = cross_val_predict(estimator, X_val, Y_val, cv=kfold, method='predict')

probabilities_test = cross_val_predict(estimator, X_test, Y_test, cv=kfold, method='predict')

scores= {}
scores['train']=r2_score(Y_train, probabilities_train)
scores['val']=r2_score(Y_val, probabilities_val)
scores['test']=r2_score(Y_test, probabilities_test)

scores

{'train': 0.8452383492534246,
 'val': 0.5076885500101797,
 'test': 0.6756645007329956}

In [14]:
from sklearn.metrics import max_error
max_errors = {}
max_errors['train'] = max_error(Y_train, probabilities_train)
max_errors['val'] = max_error(Y_val, probabilities_val)
max_errors['test'] = max_error(Y_test, probabilities_test)

max_errors

{'train': 25.53119474502796,
 'val': 28.060472700195675,
 'test': 21.85746929893586}